In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from inverse import fit_SLSQP

In [2]:
with open("data/datasets/report.npz", 'rb') as f:
    dataset = np.load(f)
    isotherm_data = dataset["isotherm_data"]
    pore_distribution_data = dataset["pore_distribution_data"]

In [ ]:
kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")[:, 77:367]
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Silica.npy")[77:367]
i = 112
res = fit_SLSQP(adsorption=isotherm_data[i], kernel=kernel, a_array=pore_widths, alpha=0)

In [ ]:
plt.plot(pore_widths, res.x, marker='.', label="SLSQP")
plt.plot(pore_widths, pore_distribution_data[i] , marker='.', label="FILE")
plt.legend()
plt.show()

In [ ]:
isotherm_by_distribution = np.dot(kernel.T, res.x)
plt.plot(pressures, isotherm_data[i], marker='.', label="original isotherm")
plt.plot(pressures, isotherm_by_distribution, marker='.', label="isotherm by distribution")
#plt.plot(pressures, np.dot(kernel.T, pore_distribution_data[i] *4000), marker='.', label="isotherm by FILE distribution")
plt.legend()
plt.show()

In [ ]:
np.sum(np.square(isotherm_by_distribution - isotherm_data[i]))

In [ ]:
alpha_arr = [0, 5, 10, 20, 30, 40]
err_res = []
i = 0
for alpha in alpha_arr:
    err = 0
    for isotherm in isotherm_data:
        #distr = fit_SLSQP(adsorption=isotherm, kernel=kernel, a_array=pore_widths, alpha=alpha).x
        isotherm_by_distribution = np.dot(kernel.T, pore_distribution_data[i])
        err += np.sum(np.square(isotherm_by_distribution - isotherm))
    err_res.append(err)

In [22]:
plt.plot(alpha_arr, err_res, marker=".")
plt.show()